In [1]:
import pandas as pd

# grep -v "^#" CEDIE1A.vcf | awk '{if ($2 > 10) print $1, $2, $2+1;  else print $1, 0, $2+1; }' |tr ' ' '\t' > CEDIE1A.vcf.bed
#run = 'coverageBed -a CEDIE1A.vcf.bed -b ~/Dropbox/UBIT/casos/caso1BZ/agilent_region.bed >testcov.tsv'
#os.system(run)
coverage = pd.read_table('/home/ariel/Projects/Gutierrez/CEDIE/201803/vcfs/testcov.tsv',header = None)

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
coverage.ix[:,6].value_counts()

0.0    563226
1.0     55319
Name: 6, dtype: int64

In [17]:
from pyensembl import EnsemblRelease as ensbl
import pandas as pd
ensemble_version = 75
data = ensbl(ensemble_version)


In [52]:
def get_bed_locus_info_from_gene_name(gene):
    exon = data.exon_ids_of_gene_name('STAT1')
    exonLocus = [data.locus_of_exon_id(e) for e in exon]

    elen =  [(e.to_dict()['start'],e.to_dict()['end']) for e in exonLocus]

    whole_gen = data.loci_of_gene_names('STAT1')
    whole_gen_len = [whole_gen[0].to_dict()['end']-whole_gen[0].to_dict()['start']]


[56602]


In [86]:
#get_locis(['STAT1','STAT4'],ref = 37,outpath='./',write_bedfile=True)
%del gen

ERROR:root:Line magic function `%del` not found.


In [ ]:
def get_locis(genelist,ref,outpath,write_bedfile = True):
    analyzed_genes = []
    try:
        if (ref == 37)|(ref == 19):
            ensemble_version = 75
    except:
        'You must to install the ensemble version 75, please run \n'        
        'pyensembl install --release 75 --species homo_sapiens \n'
    try:
        if ref == 38:
            ensemble_version = 87
    except:
        'You must to install latest ensemble version (87), please run \n'        
        'pyensembl install --release 87 --species homo_sapiens \n'
        
    data = ensbl(ensemble_version)

    ############ By the whole Gene #############
    locis = []
    for gen in genelist:
        try:
            loc =  data.loci_of_gene_names(gen)
            locis.append(loc[0].to_dict())
            analyzed_genes.append(gen)
        except:
            print 'warning, gen %s was not found and ignored'%gen
#    return pd.concat(locis)
    print '\n'
    ggenes = analyzed_genes
    
    locis = pd.DataFrame(locis)
    locis['name'] = ggenes
    locis.rename(columns = {'contig':'chrom'},inplace = True)
    locis['score'] = 0 ## this is only for completness, in order to recognize strand as the right next field
    locis = locis[['chrom','start','end','name','score','strand']]
    
    # write file
    bedfile = outpath+'gene_loci.bed'
    if write_bedfile:
        locis.to_csv(bedfile,sep = '\t',index = False,header = None)    

        
    return locis, bedfile, locis.columns



In [10]:
import sys

In [19]:
from pyensembl import EnsemblRelease as ensbl
def get_exons(genelist,ref,outpath,write_bedfile = True):
    analyzed_genes = []
    try:
        if (ref == 37)|(ref == 19):
            ensemble_version = 75
    except:
        'You must to install the ensemble version 75, please run \n'        
        'pyensembl install --release 75 --species homo_sapiens \n'
    try:
        if ref == 38:
            ensemble_version = 87
    except:
        'You must to install latest ensemble version (87), please run \n'        
        'pyensembl install --release 87 --species homo_sapiens \n'
        
    data = ensbl(ensemble_version)
    ############ By Exons #############
    exonbed = []
    for gene in genelist:
        try:
            exons = data.exon_ids_of_gene_name(gene)
            exonLocus = [data.locus_of_exon_id(e) for e in exons]
            exonLoci = [ex.to_dict() for ex in exonLocus]
            exonLoci = pd.DataFrame(exonLoci)
            exonLoci['name'] = exons
            exonLoci['gene'] = [gene]*len(exons)
            exonLoci.rename(columns = {'contig':'chrom'},inplace = True)
            exonLoci['score'] = 0 ## this is only for completness, in order to recognize strand as the right next field
            exonLoci = exonLoci[['chrom','start','end','name','score','strand','gene']]
            exonbed.append(exonLoci)
            analyzed_genes.append(gene)
        except:
            print 'warning, gene %s was not found and ignored'%gene
            sys.exc_info()[0]
#    return pd.concat(locis)
    print '\n'
    exonbed = pd.concat(exonbed)
    
    # write file
    exonbedfile = outpath+'exons_loci.bed'

    if write_bedfile:
        exonbed.to_csv(exonbedfile,sep = '\t',index = False,header = None)
    
        
    return exonbed, exonbedfile, exonbed.columns

    


In [20]:
bed, bedfile, columns = get_exons(['STAT1','STAT4'],ref = 37,outpath='./',write_bedfile=True)

In [21]:
 exome_coverage = pd.read_table('./test_output/exons_loci.bed.tsv')

In [52]:
exome_report = exome_coverage.groupby(['score'])[['coverage_dp1','len_gen_bp','bp_at_10dp','bp_at_20dp','bp_at_30dp']].sum()
rel = exome_report.apply(lambda x:x/float(x['len_gen_bp']),axis = 1).drop([u'len_gen_bp'],axis = 1)
exome_report = exome_report[['len_gen_bp']].join(rel)



In [87]:
def get_locis(genelist,ref,outpath,write_bedfile = True):
    analyzed_genes = []
    try:
        if (ref == 37)|(ref == 19):
            ensemble_version = 75
    except:
        'You must to install the ensemble version 75, please run \n'        
        'pyensembl install --release 75 --species homo_sapiens \n'
    try:
        if ref == 38:
            ensemble_version = 87
    except:
        'You must to install latest ensemble version (87), please run \n'        
        'pyensembl install --release 87 --species homo_sapiens \n'
        
    data = ensbl(ensemble_version)
    locis = []

    for gen in genelist:
        try:
            loc =  data.loci_of_gene_names(gen)
            locis.append(loc[0].to_dict())
            analyzed_genes.append(gen)
        except ValueError:
            print 'warning, gen %s was not found and ignored'%gen
#    return pd.concat(locis)
    print '\n'
    ggenes = analyzed_genes
    locis = pd.DataFrame(locis)
    locis['name'] = ggenes
    locis.rename(columns = {'contig':'chrom'},inplace = True)
    locis['score'] = 0 ## this is only for completness, in order to recognize strand as the right next field
    locis = locis[['chrom','start','end','name','score','strand']]
    
    # write file
    bedfile = outpath+'gene_loci.bed'
    if write_bedfile:
        locis.to_csv(bedfile,sep = '\t',index = False,header = None)
        
    return locis, bedfile, locis.columns


In [98]:
a,b,c =get_locis(['STAT1','STAT3','IGHM'],ref = 37, outpath = './',write_bedfile=False)

In [1]:
import mygene
from pyensembl import EnsemblRelease as ensbl
import pandas as pd




In [54]:
gene = 'IL-10Ra'.strip('')
print gene
mg = mygene.MyGeneInfo()
gene_query = mg.query(gene, size=5,species = 'human')
gids = gene_query['hits']
print  gids
if(len(gids)==0):
    print 'warning, gene %s was not found and ignored'%gene 
#    continue
else:
    gid = int(gids[0][u'_id'])
    if len(gids)>1:
        print 'warning: more than one mathch with %s'%gene
        print 'gen id: %s was assumed'%gid
loc =  data.locus_of_gene_id(gid)
locis.append(loc[0].to_dict())
analyzed_genes.append(gene)


IL-10Ra
[{u'name': u'interleukin 10 receptor subunit alpha', u'_score': 9.773872, u'symbol': u'IL10RA', u'taxid': 9606, u'entrezgene': 3587, u'_id': u'3587'}]


In [72]:
#gene[2] =='-'
gene2 = gene.replace('-', '')
gene2#.replace('-', '')

'IL10Ra'

In [40]:
#%matplotlib inline
#from matplotlib import pyplot as plt
#histogr = pd.read_csv('./byposition.txt',sep = '\t',header = None)
#plt.scatter(histogr[6],histogr[7])
#plt.scatter()

In [73]:
data = ensbl(75)
try:
    data.loci_of_gene_names('STAT1')
except ValueError:
    print 'gen no encontrado'

In [ ]:
exons = data.exon_ids_of_gene_id

In [77]:
data.locus_of_gene_id()

ValueError: No results found for query:

            SELECT distinct seqname, start, end, strand
            FROM gene
            WHERE gene_id = ?
        
with parameters: [3669]